In [16]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Reusable classes
import sys
sys.path.append("/home/jovyan/work/upload/ml-experiments/sc_wrangling")

# Data path
import pathlib
FOLDER = pathlib.Path("/home/jovyan/work/upload/")
TABLE_PATH = FOLDER / 'data/pickle/df_columns_labels.pkl'
LAW2VEC_PATH = FOLDER / 'data/law2vec/Law2Vec.200d.txt'

# Manage ML lifecycle
import mlflow
from mlflow import log_params, set_tags, log_metrics
MLFLOW_TRACKING_URI = 'http://srv.meaningfy.ws:8989'

# Math stuff
import numpy as np
from numpy.core.records import ndarray

# Data visualisation
import pandas as pd
from pandas import Series

# Vectorize algotrithm
from gensim.models import Word2Vec, KeyedVectors

# Support Vector Classifier Algorithm
from sklearn.neighbors import KNeighborsClassifier as KNC

# Intermediate steps of the pipeline must be ‘transforms’,
# that is, they must implement fit and transform methods.
from sklearn.pipeline import Pipeline

# Splitting data into train and test
from sklearn.model_selection import train_test_split
RANDOM_STATE = 42
# Where "train" contains 70% of data and "test" - 30%
TEST_SIZE = 0.3
SHUFFLE = True

# Mean embedding function
from mean_vectorizer import MeanEmbeddingVectorizer

# Metrics Evaluation Methods
from evaluation_metrics import model_evaluation_metrics

# Transform DataFrame to dictionary
from dictionary_transformation import series_pair_to_dict

In [17]:
# Read data
df = pd.read_pickle(TABLE_PATH)
df.head()

,Concatenated Data (clean),Category,Subcategory,Type of measure,Target groups,One person or microenterprises|Self-employed|Solo-self-employed,SMEs|Sector specific set of companies,One person or microenterprises|SMEs,Employees in standard employment|Other groups of workers,Employees in standard employment|Larger corporations,...,Children (minors)|Disabled|Older citizens|Parents|SMEs|Single parents|Single parents in employment|The COVID-19 risk group|Workers in care facilities|Workers in essential services,Employees in standard employment|Workers in care facilities|Workers in non-standard forms of employment,Companies providing essential services|Workers in essential services,Contractors of a company,Other businesses|Unemployed,Seasonal workers|Workers in non-standard forms of employment,Employees in standard employment|Particular professions,Businesses,Citizens,Workers
0,"[hardship, case, fund, safety, net, selfemploy...",2,12,0,One person or microenterprises|Self-employed|S...,1,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
1,"[state, support, tourism, access, financeas, t...",0,0,0,SMEs|Sector specific set of companies,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,"[bank, guarantees, smes, oneperson, enterprise...",0,0,0,One person or microenterprises|SMEs,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,"[emergency, measures, relating, shorttime, wor...",6,14,0,Employees in standard employment|Other groups ...,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1
4,"[airbus, agreement, making, unworked, hours, p...",3,7,2,Employees in standard employment|Larger corpor...,0,0,0,0,1,...,0,0,0,0,0,0,0,1,1,1


In [19]:
# Independent data
columns = df['Concatenated Data (clean)']
# Label data
subcategory = df['Subcategory']

## law2vec model

In [20]:
# Load a word2vec model stored in the C *text* format.
model = KeyedVectors.load_word2vec_format(LAW2VEC_PATH, binary=False)
l2v_dict = {w: vec for w, vec in zip(model.wv.index2word, model.wv.syn0)}

In [21]:
# Use pipes to implement steps of fit and transform method
knn_l2v = Pipeline([
    # Add the words we want to mean
    ("word2vec vectorizer", MeanEmbeddingVectorizer(l2v_dict)),
    # Use KNN algorithm
    ("KNN", KNC())])

## Train SVM Model

In [22]:
# Split data into test and train sets
# Where "train" contains 70% of data and "test" - 30%
X_train, X_test, y_train, y_test = train_test_split(
    columns, subcategory, random_state=RANDOM_STATE, test_size=TEST_SIZE, shuffle=SHUFFLE)

In [23]:
# Fit KNN model with out train data
knn_l2v.fit(X_train, y_train)

Pipeline(steps=[('word2vec vectorizer',
                 <mean_vectorizer.MeanEmbeddingVectorizer object at 0x7f7363f5fb20>),
                ('KNN', KNeighborsClassifier())])

In [24]:
# Model Evaluation
knn_l2v.score(X_train, y_train)

0.617737003058104

In [25]:
# KNN prediction based on test data
prediction = knn_l2v.predict(X_test)
prediction

array([ 4,  2,  0,  2, 20,  9,  0,  0,  0,  9,  3, 22, 14,  0,  0,  2,  9,
       33,  2,  9, 14,  0,  0, 10, 12,  2, 10,  2, 12, 20,  2, 14,  2, 10,
        0, 20, 11, 16,  2,  9, 30, 19,  1, 15, 14,  2, 30, 20,  9, 30,  4,
       10, 14,  7, 28, 14,  9, 10,  9, 14, 14, 10,  0,  2,  9, 10, 16, 14,
       20, 10,  2,  9,  0,  0,  4,  9, 12, 12, 14, 21, 16,  4,  0, 10, 10,
        0,  0, 21, 26, 10,  2,  0,  0, 32,  4, 13,  9, 10,  0, 10,  2, 29,
        2,  2,  4, 10,  0, 20,  9, 33,  2,  0,  9,  3, 22,  2,  2, 20,  0,
       14,  9, 14,  9, 12,  2, 14, 14, 13,  5, 20, 10,  0,  4, 10, 10,  0,
       10, 14,  2,  9, 19,  2,  4, 20, 33, 10, 27, 10,  2, 14,  0,  2, 20,
        4, 22,  2, 10,  0, 30, 10, 33, 21, 14,  2, 22, 29, 30,  4, 10,  9,
        2, 14, 10, 22, 20, 32, 10,  9,  7,  0, 26,  4, 14,  9,  0,  0,  0,
        8, 13,  0, 10, 33, 15,  2, 19,  4, 12, 21, 18, 20, 21, 28,  0, 14,
        2, 14, 19, 15,  0, 19, 14, 10, 10, 30,  0,  0,  2,  9, 14,  0, 10,
       16, 20,  4, 30, 10

## Evaluation

In [26]:
evaluation = model_evaluation_metrics(y_test, prediction)
evaluation

,Metric Labels,Metric values
0,Accuracy,0.441281
1,Precision,0.365845
2,Recall,0.321464
3,F1-Score,0.307104
4,Mean Absolute Error,6.476868
5,Mean Squared Error,107.722420


In [27]:
transformation = series_pair_to_dict(evaluation, 'Metric Labels', 'Metric values')
transformation

{'Accuracy': 0.4412811387900356,
 'Precision': 0.365845079677478,
 'Recall': 0.3214639176760388,
 'F1-Score': 0.3071041148071594,
 'Mean Absolute Error': 6.476868327402135,
 'Mean Squared Error': 107.72241992882562}

In [ ]:

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="Subcategory (KNN, Law2Vec)")

with mlflow.start_run():

    parameters = {"Language model": 'law2vec',
                  "Random state": RANDOM_STATE,
                  "Test size": TEST_SIZE,
                  "Shuffle": SHUFFLE}

    log_params(parameters)

    log_metrics(transformation)
